In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [67]:
dataset=pd.read_csv('heart.csv')
x=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1].values

CUSTOM PREDICTION

In [47]:
age=34
sex='M'
cpt='ATA'
rbp=150
col=150
fbs=0
ecg='Normal'
mer=145
ea='N'
op=1
st='down'
l=[age,sex,cpt,rbp,col,fbs,ecg,mer,ea,op,st]

In [48]:
print(x[len(x)-1])

[38 'M' 'NAP' 138 175 0 'Normal' 173 'N' 0.0 'Up']


In [49]:
print(len(x))

918


ADDING TO ARRAY

In [50]:
x=pd.DataFrame(x)
x.loc[len(x)]=l
x=np.array(x)

In [51]:
print(len(x))

919


In [52]:
print(x[len(x)-1])

[34 'M' 'ATA' 150 150 0 'Normal' 145 'N' 1 'down']


In [53]:
dataset.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [68]:
dataset.ChestPainType.value_counts()

ChestPainType
ASY    496
NAP    203
ATA    173
TA      46
Name: count, dtype: int64

In [54]:
print(x[0])

[40 'M' 'ATA' 140 289 0 'Normal' 172 'N' 0.0 'Up']


In [55]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1,2,6,8,10])],remainder='passthrough')
x=np.array(ct.fit_transform(x))

In [56]:
l=x[len(x)-1]
print(l)

[0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 34 150 150 0
 145 1]


In [57]:
print(x[len(x)-2])

[0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 38 138 175 0
 173 0.0]


In [58]:
x=pd.DataFrame(x)
x=x.drop(len(x)-1)
x=np.array(x)

In [59]:
print(x[len(x)-1])

[0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 38 138 175 0
 173 0.0]


In [60]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [61]:
print(x_test[len(x_test)-1])

[0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 53 160 0 1
 122 0.0]


In [62]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)

In [63]:
from catboost import CatBoostClassifier
cb=CatBoostClassifier(random_state=0)
cb.fit(x_train,y_train)

Learning rate set to 0.009028
0:	learn: 0.6847324	total: 6.06ms	remaining: 6.05s
1:	learn: 0.6781245	total: 18ms	remaining: 9s
2:	learn: 0.6704850	total: 27.2ms	remaining: 9.05s
3:	learn: 0.6626286	total: 35.1ms	remaining: 8.74s
4:	learn: 0.6547032	total: 41.9ms	remaining: 8.35s
5:	learn: 0.6480062	total: 49.4ms	remaining: 8.18s
6:	learn: 0.6410208	total: 64.1ms	remaining: 9.1s
7:	learn: 0.6334106	total: 70.5ms	remaining: 8.74s
8:	learn: 0.6280986	total: 77.9ms	remaining: 8.58s
9:	learn: 0.6222711	total: 83.7ms	remaining: 8.29s
10:	learn: 0.6155719	total: 93.4ms	remaining: 8.4s
11:	learn: 0.6093565	total: 99.5ms	remaining: 8.19s
12:	learn: 0.6023098	total: 105ms	remaining: 7.94s
13:	learn: 0.5961751	total: 112ms	remaining: 7.9s
14:	learn: 0.5899001	total: 118ms	remaining: 7.74s
15:	learn: 0.5837805	total: 124ms	remaining: 7.65s
16:	learn: 0.5776386	total: 130ms	remaining: 7.52s
17:	learn: 0.5720170	total: 134ms	remaining: 7.33s
18:	learn: 0.5665224	total: 140ms	remaining: 7.25s
19:	lea

In [64]:
from sklearn.metrics import accuracy_score,confusion_matrix
y_pred=cb.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_pred,y_test)*100

[[64 13]
 [10 97]]


87.5

In [65]:
from sklearn.model_selection import cross_val_score
cvs=cross_val_score(X=x_train,y=y_train,estimator=cb,cv=10)
print(cvs.mean()*100)
print(cvs.std()*100)

Learning rate set to 0.008627
0:	learn: 0.6851402	total: 4.16ms	remaining: 4.16s
1:	learn: 0.6791863	total: 9.44ms	remaining: 4.71s
2:	learn: 0.6723067	total: 16.8ms	remaining: 5.57s
3:	learn: 0.6645627	total: 21.3ms	remaining: 5.29s
4:	learn: 0.6571768	total: 25.8ms	remaining: 5.14s
5:	learn: 0.6504730	total: 32.3ms	remaining: 5.34s
6:	learn: 0.6438977	total: 36.5ms	remaining: 5.17s
7:	learn: 0.6366311	total: 43ms	remaining: 5.33s
8:	learn: 0.6313797	total: 50.8ms	remaining: 5.59s
9:	learn: 0.6250621	total: 55.3ms	remaining: 5.48s
10:	learn: 0.6183226	total: 61.4ms	remaining: 5.52s
11:	learn: 0.6120431	total: 66.7ms	remaining: 5.49s
12:	learn: 0.6045511	total: 74.4ms	remaining: 5.65s
13:	learn: 0.5986077	total: 81.1ms	remaining: 5.71s
14:	learn: 0.5927443	total: 85ms	remaining: 5.58s
15:	learn: 0.5869187	total: 89.3ms	remaining: 5.49s
16:	learn: 0.5811341	total: 95.6ms	remaining: 5.53s
17:	learn: 0.5756717	total: 99.9ms	remaining: 5.45s
18:	learn: 0.5703555	total: 104ms	remaining: 5.3

PREDICTING THE CUSTOM INPUT

In [66]:
print(cb.predict(l))

1
